In [21]:
import time
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

batch_size=1000
test_batch_size=1000
epochs=1000
device=torch.device("cpu")
device=torch.device("cuda:0")
data_path='../../../data'
kwargs = {'num_workers': 1, 'pin_memory': True} if device != torch.device('cpu') else {}
train_loader = torch.utils.data.DataLoader( 
    datasets.MNIST(data_path, train=True,download=True, 
    transform=transforms.Compose([ transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)) ])), batch_size=batch_size,
    shuffle=False, **kwargs)
test_loader = torch.utils.data.DataLoader(datasets.MNIST(data_path, train=False,
    transform=transforms.Compose([ transforms.ToTensor(), transforms.Normalize((0.1307,),
    (0.3081,)) ])), batch_size=test_batch_size, shuffle=False, **kwargs)
train_data=[]
test_data=[]
for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    train_data.append((data,target))
for batch_idx, (data, target) in enumerate(test_loader):
    data, target = data.to(device), target.to(device)
    test_data.append((data,target))

In [24]:
n_in=784
n_out=10
n_hidden=500
model = "lr" # logistic regression
model = "fc2" # 2-layer MLP
model = "conv" # convolution
if (model == "lr"):
    net = nn.Sequential(nn.Linear(n_in,n_out))
elif(model == "fc2"):
    net = nn.Sequential(nn.Linear(n_in,n_hidden),nn.Sigmoid(),nn.Linear(n_hidden,n_out))
elif(model == "conv"):
    net = nn.Sequential(nn.Conv2d(1,20,5),nn.ReLU(),nn.MaxPool2d(2),nn.Conv2d(20,50,5),nn.ReLU(),nn.MaxPool2d(2))
    net2 = nn.Sequential(nn.Linear(4*4*50,n_hidden),nn.ReLU(),nn.Linear(n_hidden,n_out))
else:
    print("Wrong model")
net = net.to(device)
net2 = net2.to(device)
optimizer = torch.optim.Adam(list(net.parameters()), lr=0.001, betas=(0.9, 0.999))
print_bin = epochs/100
for epoch in range(1, epochs + 1):
    optimizer.zero_grad()
    t1=time.time()
    for batch_idx, (data, target) in enumerate(train_data):
        if(model == 'conv'):
            output=net(data).view(output.shape[0],-1)
            output = net2(output)
        else:
            output=net(data.view(data.shape[0],-1))

        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
    test_loss = 0
    correct = 0
    for data, target in test_data:
        if(model == 'conv'):
            output=net(data).view(output.shape[0],-1)
            output = net2(output)
        else:
            output=net(data.view(data.shape[0],-1))
        output=F.log_softmax(output, dim=1)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= 10000
    correct /= 10000
    if(epoch % print_bin==0):
        print("#%d loss=%.3f test_loss=%.3f accuracy=%.3f time=%.3f"%(epoch,loss,test_loss,correct,time.time()-t1))
        

#10 loss=0.231 test_loss=0.266 accuracy=0.923 time=0.419
#20 loss=0.227 test_loss=0.257 accuracy=0.922 time=0.418
#30 loss=0.492 test_loss=0.577 accuracy=0.828 time=0.418
#40 loss=0.124 test_loss=0.097 accuracy=0.972 time=0.418
#50 loss=0.108 test_loss=0.076 accuracy=0.977 time=0.418
#60 loss=0.191 test_loss=0.168 accuracy=0.948 time=0.418
#70 loss=0.122 test_loss=0.072 accuracy=0.978 time=0.419
#80 loss=0.120 test_loss=0.075 accuracy=0.978 time=0.418
#90 loss=0.224 test_loss=0.203 accuracy=0.938 time=0.418
#100 loss=0.097 test_loss=0.058 accuracy=0.981 time=0.418
#110 loss=0.094 test_loss=0.059 accuracy=0.983 time=0.418
#120 loss=0.099 test_loss=0.051 accuracy=0.985 time=0.418
#130 loss=0.206 test_loss=0.182 accuracy=0.949 time=0.418
#140 loss=0.097 test_loss=0.050 accuracy=0.984 time=0.417
#150 loss=0.322 test_loss=0.418 accuracy=0.905 time=0.417
#160 loss=0.096 test_loss=0.053 accuracy=0.985 time=0.417
#170 loss=0.100 test_loss=0.051 accuracy=0.985 time=0.418
#180 loss=0.112 test_lo

KeyboardInterrupt: 